In [1]:
from IPython.display import clear_output 

In [2]:
!pip3 install mayavi
!pip install clifford

#!jupyter nbextension enable  --py mayavi --user

#!jupyter nbextension install  --py mayavi --user
#!jupyter notebook --generate-config

clear_output()

In [3]:
import numpy as np
import mayavi
from mayavi import mlab

from numpy import arange, pi, cos, sin, exp

from traits.api import HasTraits, Range, Instance, \
        on_trait_change
from traitsui.api import View, Item, Group

from mayavi.core.api import PipelineBase
from mayavi.core.ui.api import MayaviScene, SceneEditor, \
                MlabSceneModel


In [4]:
import clifford 
from clifford.g3c import *

In [5]:
R12 = lambda lam : exp(-e12*pi*lam)
R23 = lambda lam : exp(-e23*pi*lam)
R31 = lambda lam : exp(e13*pi*lam)

In [6]:
class Strings:
    def __init__(self,elems):
        self.vector = elems[0]
        self.beads  = len(elems)

        self.elems = elems

    def s(self, Rotor): #Sandwich product

        return Strings(Rotor*self.elems*~Rotor)


class Cubes:
    def __init__(self,elems):
        
        self.elems = elems

    def s(self, Rotor): #Sandwich product

        return Cubes(Rotor*self.elems*~Rotor)

Cube = Cubes( np.array([
                 e1 + e2 + e3,
                 e1 - e2 + e3,
                -e1 - e2 + e3,
                -e1 + e2 + e3,
                 e1 + e2 - e3,
                 e1 - e2 - e3,
                -e1 - e2 - e3,
                -e1 + e2 - e3,]))
M = 80
spacing = 0.1

String = Strings(spacing*np.arange(1/spacing,M+1)*e1)
N = 5

StringX1 = String
StringX2 = String.s(R12(0.5))

StringY1 = String.s(R12(0.25))
StringY2 = String.s(R12(0.75))

StringZ1 = String.s(R23(0.75)*R12(0.75))
StringZ2 = String.s(R23(0.25)*R12(0.75))

StringList = np.array([[StringX1,StringY1,StringZ1],
                       [StringX2,StringY2,StringZ2]])


In [7]:
def W(lam,rotation_axis,string_axis,String):
    '''
    string_axis i : axis on which string is attached 1,2,3 
    rotation_axis k : rotation axis 1,2,3

    returns rotor
    '''

    to_vector = {1 : e1, -1 : -e1,
                 2 : e2, -2 : -e2,
                 3 : e3, -3 : -e3}

    ei = to_vector[string_axis]
    ek = to_vector[rotation_axis]

    ej = -(ei ^ ek) / e123
    

    Rij = lambda lam : exp(-ei^ej*pi*lam) #around rot axis
    Rjk = lambda lam : exp(-ej^ek*pi*lam)

    
    lam %= 2

    M = len(String.elems)+1
    RotString = np.zeros_like(String.elems)

    a = np.arange(2,M+1) #index along the string
    alpha = a / (M-1)
    
    """
    lambda_ij = (lam      if 0 <= lam and lam < 0.5 else 
                        1 - lam  if 0.5 <= lam <= 1 else
                        lam-1     if 1 < lam <= 1.5 else
                        2-lam) * np.maximum(1-alpha**3,0)

    lambda_jk = 0.5*(lam  if 0<=lam and lam < 1 else
                            1-lam) 
    """

    lambda_ij = (lam      if 0 <= lam and lam < 0.5 else 
                        1 - lam  if 0.5 <= lam <= 1 else
                        1-lam   if 1 < lam <= 1.5 else
                        lam-2 ) 

    lambda_jk = 0.5*(lam  if 0<=lam and lam < 1 else
                            2-lam)    

    
    
    RotString = String.s(Rjk(lambda_jk )
                         * Rij(lambda_ij *(1-alpha**3))    #perpendicular to rotation plane
                                    #along rotation axis
                         ) 
    


    return RotString



R = [R23,R31,R12]

def rotors(a,b,lam,StringList):
    '''
    a : quat current state 1,2,3
    b : quat rotation 1,2,3
    lam : rotation number
    StringList : 3-tuple of strings
    '''
    sign = +1
    if a < 0:
        a = -a
        sign = -1

    a -= 1; b -= 1
    a %= 3; b %= 3

    if b == (a+1) % 3:
        c = (a-1) % 3
        phase = -1
    else:
        c = (a+1) % 3
        phase = +1

    StringList = [StringList[a], StringList[b], StringList[c]]

    RotStringList = [W(lam + 1 + (1 if sign<0 else 0), b+1, a+1, StringList[0]), 
                     W(3/2,a+1,b+1,StringList[1]).s(R[b](lam/2+1/4 + (1/2 if sign<0 else 0))), 
                     W(lam + phase*1/2 + (1 if sign<0 else 0), b+1, c+1, StringList[2])]
        
    RotStringList_ = [0,0,0]
    RotStringList_[a] = RotStringList[0] 
    RotStringList_[b] = RotStringList[1]
    RotStringList_[c] = RotStringList[2]
    
    return RotStringList_

to_symbol = {1 : "i", -1 : "-i",
             2 : "j", -2 : "-j",
             3 : "k", -3 : "-k"
             }

#plt.rcParams['figure.figsize'] = [4*N, 4]
#fig, axs = plt.subplots(1,N,subplot_kw=dict(projection='3d'))


StringList1 = [StringX1,StringY1,StringZ1]
StringList2 = [StringX2,StringY2,StringZ2]


In [8]:
up_ = np.array([
    +e1+e2+e3,
    -e1+e2+e3,
    +e1-e2+e3,
    -e1-e2+e3,
])  
down_ = np.array([
    +e1+e2-e3,
    -e1+e2-e3,
    +e1-e2-e3,
    -e1-e2-e3,
]) 
front = np.array([
    +e1+e2+e3,
    -e1+e2+e3,
    +e1+e2-e3,
    -e1+e2-e3,
]) 
back = np.array([
    +e1-e2+e3,
    -e1-e2+e3,
    +e1-e2-e3,
    -e1-e2-e3,
]) 
left = np.array([
    -e1+e2+e3,
    -e1-e2+e3,
    -e1+e2-e3,
    -e1-e2-e3,
]) 
right = np.array([
    +e1+e2+e3,
    +e1-e2+e3,
    +e1+e2-e3,
    +e1-e2-e3,
]) 

faces = np.array([
    right,front, up_,
    left, back,  down_,
])

In [9]:
faces = faces.reshape((6,2,2))

In [10]:
Cube = Cubes(faces)

In [12]:
def cube_faces(lam):
    
    b = 3 #axis of rot

    RotCube = Cube.s(R[abs(b)-1](lam))
    
    return RotCube.elems

def string_points(lam):
    a = -1 #state
    b = 3 #axis of rot
    
    ######## 
    #sampling later and interpolate before? ?? 
    #
    ########
    RotStrings1 = rotors(a,b,lam,StringList1) #pos coor dir
    RotStrings2 = rotors(a,b,lam,StringList2) #neg coor dir
    
    
    return RotStrings1, RotStrings2


class MyModel(HasTraits):
    lam    = Range(-0.5, 0.5, 0.0)  # mode='spinner')

    scene = Instance(MlabSceneModel, ())
    
    #mult plot instances for mult refeshes?
    face_plot1 = Instance(PipelineBase)
    face_plot2 = Instance(PipelineBase)
    face_plot3 = Instance(PipelineBase)
    face_plot_1 = Instance(PipelineBase)
    face_plot_2 = Instance(PipelineBase)
    face_plot_3 = Instance(PipelineBase)
    
    axis_plot1 = Instance(PipelineBase)
    axis_plot2 = Instance(PipelineBase)
    axis_plot3 = Instance(PipelineBase)
    axis_plot_1 = Instance(PipelineBase)
    axis_plot_2 = Instance(PipelineBase)
    axis_plot_3 = Instance(PipelineBase)
    
        
    # When the scene is activated, or when the parameters are changed, we
    # update the plot.

    
    @on_trait_change('lam,scene.activated')
    def update_plot(self):
        #mlab.clf()
        
        faces = cube_faces(self.lam)
        axes1,axes2 = string_points(self.lam)
        
        colors = ((1,0,0),(0,1,0),(0,0,1))
        
        self.face_plot1; self.face_plot2; self.face_plot3
        self.face_plot_1; self.face_plot_2; self.face_plot_3
        
        self.axis_plot1; self.axis_plot2; self.axis_plot3
        self.axis_plot_1; self.axis_plot_2; self.axis_plot_3
        
        ##### --> iter through self plot attributes 
        face_plots = ["face_plot1","face_plot2","face_plot3",
                      "face_plot_1","face_plot_2","face_plot_3"]
        axis_plots1 = ["axis_plot1","axis_plot2","axis_plot3"]
        axis_plots2 = ["axis_plot_1","axis_plot_2","axis_plot_3"]
        
        """
        ###
        
        
        x,y,z = (np.array(faces[0]|e1,dtype=float),
                 np.array(faces[0]|e2,dtype=float),
                 np.array(faces[0]|e3,dtype=float))
        
        if self.face_plot1 is None:
            self.face_plot1 = self.scene.mlab.mesh(x,y,z,opacity=1.0, color = colors[0])
        else:
            self.face_plot1.mlab_source.trait_set(x=x, y=y, z=z) 
         
            
        x,y,z = (np.array(axes1[0].elems|e1,dtype=float),
                 np.array(axes1[0].elems|e2,dtype=float),
                 np.array(axes1[0].elems|e3,dtype=float))

        if self.axis_plot1 is None:
            self.axis_plot1 = self.scene.mlab.plot3d(x, y, z, color = colors[0],tube_radius = 0.2)
        else:
            self.axis_plot1.mlab_source.trait_set(x=x, y=y, z=z) 
            
        """
        for face_plot,face,color in zip(face_plots,faces,2*colors): #gives rotated face elems
            x,y,z = (np.array(face|e1,dtype=float),
                     np.array(face|e2,dtype=float),
                     np.array(face|e3,dtype=float))
                
            if vars(self)[face_plot] is None:
                vars(self)[face_plot] = self.scene.mlab.mesh(x,y,z,opacity=1.0, color = color)
            else:
                vars(self)[face_plot].mlab_source.trait_set(x=x, y=y, z=z) 
        
        for axis_plot,points,color in zip(axis_plots1,axes1,colors):
            points     = points.elems


            x,y,z = (np.array(points|e1,dtype=float),
                     np.array(points|e2,dtype=float),
                     np.array(points|e3,dtype=float))

            if vars(self)[axis_plot] is None:
                vars(self)[axis_plot] = self.scene.mlab.plot3d(x, y, z, color = color,tube_radius = 0.2)
            else:
                vars(self)[axis_plot].mlab_source.trait_set(x=x, y=y, z=z) 
        
        
        for axis_plot,points,color in zip(axis_plots2,axes2,colors):
            points     = points.elems

            x,y,z = (np.array(points|e1,dtype=float),
                     np.array(points|e2,dtype=float),
                     np.array(points|e3,dtype=float))

            if vars(self)[axis_plot] is None:
                vars(self)[axis_plot] = self.scene.mlab.plot3d(x, y, z, color = color,tube_radius = 0.2)
            else:
                vars(self)[axis_plot].mlab_source.trait_set(x=x, y=y, z=z) 

        ###
        
        
    # The layout of the dialog created
    view = View(Item('scene', editor=SceneEditor(scene_class=MayaviScene),
                     height=500, width=600, show_label=False),
                Group(
                        '_', 'lam',
                     ),
                resizable=True,
                )

my_model = MyModel()
my_model.configure_traits()

True